In [1]:
import os
import requests
import json
import pandas as pd
import time


def get_questions(previous_id, count=100):
    base_url = "https://rulesguru.net/api/questions/"
    parameters = {
        "count": count,
        "level": ["0", "1", "2", "3", "Corner Case"],
        "complexity": ["Simple", "Intermediate", "Complicated"],
        "legality": "All of Magic",
        "tags": [],
        "previousId": int(previous_id),
    }

    response = requests.get(base_url, params={"json": json.dumps(parameters)})
    response.raise_for_status()
    return response.json()


if os.path.exists("data/rules_guru/rules_guru_qa_dataset.csv"):
    df = pd.read_csv("data/rules_guru/rules_guru_qa_dataset.csv")
    previous_id = df["id"].max()
else:
    df = pd.DataFrame()
    previous_id = 1

count = 100
while True:
    time.sleep(3)
    questions = get_questions(previous_id, count=count)
    if questions["status"] != 200:
        if (
            questions["status"] == 400
            and questions["error"] == "Incorrectly formatted json."
        ):
            count = count // 2
            print(f"Reducing count ({count})...")

            if count < 10:
                previous_id += 1
                count = 100

            continue

        print("ERROR:")
        print(questions)
        break
    else:
        if count < 100:
            print(f"Restoring count to 100...")
            count = 100

    if len(df) == 0:
        df = pd.DataFrame(questions["questions"])
    else:
        df = pd.concat([df, pd.DataFrame(questions["questions"])])
        df = df.drop_duplicates(subset="id", keep="first")

    previous_id = df["id"].max()

    df.to_csv("data/rules_guru/rules_guru_qa_dataset.csv", index=False)

    print(f"Downloaded {len(df)} questions / ID {previous_id}")

Downloaded 1457 questions / ID 7357
ERROR:
{'status': 429, 'error': "Please don't send more than one request every 2 seconds."}


In [2]:
import pandas as pd
from gatherer_sage.utils import clean_text

df = pd.read_csv("data/reddit/reddit_qa_dataset_with_context.csv").dropna()

In [3]:
for c in ["question", "answer", "context"]:
    print(c, df[c].apply(clean_text).str.contains("&gt;").sum())

question 0
answer 0
context 0
